In [ ]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
import glob
import os
import cv2
import numpy
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Conv2D
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from keras.layers import Dense, Activation, Flatten
import tensorflow as tf
from numpy import asarray

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

plt.style.use('dark_background')

from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from keras.utils import normalize, to_categorical
# import splitfolders

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
splitfolders.ratio('/content/drive/MyDrive/Dataset', output = '/content/drive/MyDrive/full',
                    ratio = (0.8 , 0.0 , 0.2)) 


Copying files: 6400 files [02:38, 40.51 files/s] 


In [ ]:
size = 128
train_images = []
train_labels = []
for directory_path in glob.glob("/content/drive/MyDrive/full/train/*"):
    label = directory_path.split("\\")[-1]
    print('\n')
    # print(label)
    print('\n')
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        print(img_path)
        img = cv2.imread(img_path,cv2.IMREAD_UNCHANGED)
        img = cv2.resize(img,(size,size))
        train_images.append(img)
        train_labels.append(label)
        

"""Note: Better to work with (numpy) array than a normal list as in the normal
list we will have to iterate over all the elements using a loop
"""


# print('\n')
# print("Train images and Train labels shape: ")
# print("Shape: ", train_images.shape)
# print("Shape: ", train_labels.shape)
# print('\n')
train_images = np.array(train_images)
train_labels = np.array(train_labels)
print(train_labels)

In [ ]:
test_images = []
test_labels = []
for directory_path in glob.glob("/content/drive/MyDrive/full/test/*"):
    label = directory_path.split("\\")[-1]
    print('\n')
    print(label)
    print('\n')
    for img_path in glob.glob(os.path.join(directory_path, '*.jpg')):
        print(img_path)
        img = cv2.imread(img_path,cv2.IMREAD_UNCHANGED)
        img = cv2.resize(img,(size,size))
        test_images.append(img)
        test_labels.append(label)

# print('\n')
#print("Test images and Test labels shape: ")
#print('\n')
#print('Test Images: ' ,test_images.shape)
print(type(test_images))
test_images = np.array(test_images, dtype =  object)
test_labels = np.array(test_labels)
print(type(test_images))

In [ ]:
'''
Here we encoded the labels and the reason behind that is because
converting the labels into a numeric form so as to convert them 
into the machine-readable form will be easier better model and 
most algorithms cannot handle  the "Strings" unless they are 
converted into a numerical value.
'''
# print(type(train_labels))
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(test_labels)
test_labels_encoded = le.transform(test_labels)
le.fit(train_labels)
train_labels_encoded = le.transform(train_labels)
print(train_labels_encoded)
# print(type(train_labels))

[1 1 1 ... 2 2 2]


In [ ]:
'''
Here we reassingned the variables to new names as they are more meaningful(That is not actual splitting)
The splitting was already done
'''
x_train, y_train, x_test, y_test = train_images, train_labels_encoded, test_images, test_labels_encoded

In [ ]:
'''
Each number represents a color code. When using the 
image as it is and passing through a Deep Neural Network,
the computation of high numeric values may become more complex.
To reduce this we can normalize the values to range from 0 to 1.
'''

x_train = x_train / 255.0
x_test = x_test / 255.0



In [ ]:
from keras.utils import to_categorical
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)
print(train_images.shape)

(5119, 128, 128)


In [ ]:
#we could use other activation functions to see the best accuracy
activation = 'sigmoid'

#sequential(), in keras it is used to arrange Keras layers in sequntial order, until it reaches the output layer
feature_extractor = Sequential()
'''
Conv2D, it is considerd a filter or a kernel used for 2d input data
'''
feature_extractor.add(Conv2D(32, (3,3), activation = activation, padding = 'same', input_shape = (size, size, 1)))
feature_extractor.add(BatchNormalization())

feature_extractor.add(Conv2D(32, (3,3), activation = activation, padding = 'same', kernel_initializer = 'he_uniform'))

'''BatchNormalization(), used in between the  CNN layers,
   througth segmenting data into mini_batches so the learning
   process would be faster.
'''
feature_extractor.add(BatchNormalization())
feature_extractor.add(MaxPooling2D())

feature_extractor.add(Conv2D(64, (3,3), activation = activation, padding = 'same', kernel_initializer = 'he_uniform'))
feature_extractor.add(BatchNormalization())

feature_extractor.add(Conv2D(64, (3,3), activation = activation, padding = 'same', kernel_initializer = 'he_uniform'))
feature_extractor.add(BatchNormalization())
feature_extractor.add(MaxPooling2D())

feature_extractor.add(Flatten())

In [ ]:
x = feature_extractor.output  
x = Dense(128, activation = activation, kernel_initializer = 'he_uniform')(x)
prediction_layer = Dense(4, activation = 'softmax')(x)

In [ ]:
# Make a new model combining both feature extractor and x
cnn_model = Model(inputs=feature_extractor.input, outputs=prediction_layer)
cnn_model.compile(optimizer='rmsprop',loss = 'categorical_crossentropy', metrics = ['accuracy'])
print(cnn_model.summary()) 

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_input (InputLayer)   [(None, 128, 128, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 128, 128, 32)      320       
                                                                 
 batch_normalization (BatchN  (None, 128, 128, 32)     128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 32)      9248      
                                                                 
 batch_normalization_1 (Batc  (None, 128, 128, 32)     128       
 hNormalization)                                                 
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 32)       0     

In [ ]:
history = cnn_model.fit(x_train, y_train_one_hot, epochs=5)

Epoch 1/5
160/160 [==============================] - 519s 3s/step - loss: 1.2101 - accuracy: 0.4733
Epoch 2/5
160/160 [==============================] - 516s 3s/step - loss: 1.0398 - accuracy: 0.5001
Epoch 3/5
160/160 [==============================] - 519s 3s/step - loss: 1.0388 - accuracy: 0.5001
Epoch 4/5
160/160 [==============================] - 514s 3s/step - loss: 1.0390 - accuracy: 0.4989
Epoch 5/5
160/160 [==============================] - 516s 3s/step - loss: 1.0401 - accuracy: 0.4999


In [ ]:
loss = history.history['loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()